In [2]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader('speech.txt')
data=loader.load()
data

[Document(metadata={'source': 'speech.txt'}, page_content="FAISS (Facebook AI Similarity Search), büyük ölçekli vektör arama yapabilen bir kütüphanedir. Yüksek boyutlu vektörleri hızlı bir şekilde aramak ve benzerlik ölçümü yapmak için optimize edilmiştir.\nÖrneğin, bir metin kümesini vektörlere dönüştürerek, en benzer metinleri bulmak için FAISS kullanılabilir. İlk adım olarak, metinleri belirli bir gömme (embedding) modeli ile vektörlere dönüştürmek gerekir. Daha sonra, bu vektörler FAISS indeksine eklenir ve benzerlik araması yapılır.\nFAISS'in temel özellikleri şunlardır:\nYüksek verimli vektör arama\nGPU ve CPU optimizasyonları\nÇeşitli indeksleme yöntemleri\nÖrnek bir kullanım senaryosu şu şekildedir:\nMetinleri vektörlere dönüştür\nFAISS ile bir indeks oluştur ve vektörleri ekle\nBir sorgu vektörü ile en yakın komşuları bul\nFAISS, büyük veri kümelerinde benzer öğeleri hızlıca bulmak için güçlü bir araçtır.\nAma FAISS doğrudan cevap oluşturmaz, sadece en benzer metin parçalarını

In [4]:
#Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50,chunk_overlap=10)
splits = text_splitter.split_documents(data)

##Embed
modelName = 'llama3.2:1b'
embedding = OllamaEmbeddings(model=modelName)

In [5]:
## Vector DB
chroma = Chroma.from_documents(documents=splits, embedding=embedding)
chroma

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

In [ ]:
## querying
query='''FAISS'in temel özellikleri nelerdir?'''
docs = chroma.similarity_search(query, k=3)
docs

[Document(id='32c0065a-539b-47eb-8bff-5339679f90ae', metadata={'source': 'speech.txt'}, page_content="FAISS'in temel özellikleri şunlardır:"),
 Document(id='6f79a23a-60bc-4fbf-9dcb-b6f7682a3684', metadata={'source': 'speech.txt'}, page_content="FAISS'in temel özellikleri şunlardır:"),
 Document(id='75063304-38cc-418d-8df6-4cd4e4cdbdba', metadata={'source': 'speech.txt'}, page_content='sadece en benzer metin parçalarını getirir.')]

In [ ]:
## saving to the disk
## Vector DB
persist_directory = './chroma'
chroma = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=persist_directory) # or chroma.save(persist_directory)
chroma

In [ ]:
# load from disk
chroma2 = Chroma(persist_directory=persist_directory,embedding_function=embedding)
chroma2

In [ ]:
docs = chroma2.similarity_search_with_score(query, k=3)
docs

[(Document(id='58ceb353-462b-477f-ae06-805b2ba139ab', metadata={'source': 'speech.txt'}, page_content="FAISS'in temel özellikleri şunlardır:"),
  0.31311859881279513),
 (Document(id='9664b794-5d0a-48d7-b692-1708a33e08bd', metadata={'source': 'speech.txt'}, page_content='sadece en benzer metin parçalarını getirir.'),
  0.7441930856059233),
 (Document(id='3d994ae9-9b42-4a9f-bec5-565250fc8256', metadata={'source': 'speech.txt'}, page_content='Örnek bir kullanım senaryosu şu şekildedir:'),
  0.7494630934148299)]

In [ ]:
### Retriever option
retriever = chroma2.as_retriever()
retriever.invoke(query, k=3)

NameError: name 'chroma2' is not defined